<a href="https://colab.research.google.com/github/mqifjjfwiot/boardgame/blob/main/sample_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Libarary 불러오기 및 경로설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from tqdm import tqdm
import torchvision

In [ ]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
model_save_path = "/content/drive/MyDrive/Colab Notebooks/보드게임/model"
test_dir = "/content/drive/MyDrive/Colab Notebooks/보드게임/test"
save_dir = "/content/drive/MyDrive/Colab Notebooks/보드게임"
os.makedirs(model_save_path, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(save_dir, exist_ok=True)

## 1. Model 정의

## 2. Test Dataset 정의

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
def make_test_with_label(test_dir, predict_col=True):
  test_dict = {}
  test_dict['path'] = []
  test_dict['label'] = []
  columns = ["path", "label"]
  if predict_col:
    test_dict['predict'] = []
    columns.append("predict")

  for idx, card in enumerate(os.listdir(test_dir)):
    card_names = card.split(".")
    if card_names[-1] != 'jpg':
      continue
    label = card_names[-2].split("label")[-1]
    card_path = os.path.join(test_dir, card)
    test_dict['path'].append(card_path)
    test_dict['label'].append(int(label))
    if predict_col:
      test_dict['predict'].append("")

  return test_dict, columns

In [ ]:
def make_test_without_label(test_dir, predict_col=True):
  test_dict = {}
  test_dict['path'] = []
  columns = ["path"]
  if predict_col:
    test_dict['predict'] = []
    columns.append("predict")

  for idx, card in enumerate(os.listdir(test_dir)):
    card_names = card.split(".")
    if card_names[-1] != 'jpg':
      continue
    card_path = os.path.join(test_dir, card)
    test_dict['path'].append(card_path)
    if predict_col:
      test_dict['predict'].append("")

  return test_dict, columns


In [ ]:
test_dict, test_columns = make_test_with_label(test_dir)
print(len(test_dict["path"]), len(test_dict["label"]))

In [ ]:
test_df = pd.DataFrame(test_dict, columns=test_columns)

In [ ]:
test_df.head()

In [ ]:
submission = test_df.copy()
transform = transforms.Compose([
    # transforms.CenterCrop((320, 256)),
    transforms.Resize((256, 256), Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
BATCH_SIZE = 16

In [ ]:
dataset = TestDataset(test_df, transform)

loader = DataLoader(
    dataset,
    shuffle=False,
    batch_size=BATCH_SIZE, 
    num_workers=1
)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
OUTPUT = 3
model = torchvision.models.resnet18()
model.fc = torch.nn.Linear(in_features=512, out_features=OUTPUT, bias=True)
model.load_state_dict(torch.load(os.path.join(model_save_path, f"5.pt")))

In [ ]:
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장
all_predictions = []
for (images, label) in tqdm(loader):
    with torch.no_grad():
        # pred = model.forward(images.to(device))
        # _, y_pred = torch.max(pred, 1)
        # all_predictions.extend(y_pred.cpu().numpy())
        
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())

## 3. Inference

In [ ]:
print(len(all_predictions))

test inference is done!


In [ ]:
submission['predict'] = all_predictions

# 예측 결과 파일 저장
submission.to_csv(os.path.join(save_dir, 'submission.csv'), index=False, encoding="utf-8-sig")
print('test inference is done!')

In [ ]:
submission.head()